# **Interactive Map with PACE and insitu data**<br>
Author:Gulce Kurtay<br>
ORCID: 0000-0002-5169-6314<br>
Sources:<br>
hypercoast, https://hypercoast.org/<br>
Woods Hole Dashboard, https://ifcb-data.whoi.edu/dashboard <br>

This notebook designed to combine PACE chl-a data (hypercoast) with in-situ flow cytometry group distribution then make an interactive map.<br>
Classified IFCB data is limited, so make sure to search the database then find a group distribution that matches with your interest. 



STEP 2.2 summarize the information into daily format to match up with PACE data <br>
STEP 3   Download the PACE data with hypercoast<br>
STEP 4   interactive map with spatial chl-a and insitu group distribution


##STEP 3  Download the PACE data by using Hypercoast package

In [ ]:
#first install the hypercoast
%pip install "hypercoast[extra]"

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import hypercoast
from hypercoast import Map

In [2]:
# Now load the data
import pandas as pd
difcb=pd.read_csv("../../data/mvco_2024.csv")
# Convert the cleaned date strings to datetime objects
difcb['date'] = pd.to_datetime(difcb['date'], format='%Y%m%d')
difcb.head()

,pid,max_value_column,max_value,date,pid_date,bin_id,latitude,longitude,bin_date
0,D20240215T150055_IFCB010_00001,nanoplankton_mix,0.7495,2024-02-15,D20240215T150055,D20240215T150055_IFCB010,41.325,-70.5667,D20240215T150055
1,D20240215T150055_IFCB010_00002,fiber,0.9995,2024-02-15,D20240215T150055,D20240215T150055_IFCB010,41.325,-70.5667,D20240215T150055
2,D20240215T150055_IFCB010_00003,Rhizosolenia,0.9980,2024-02-15,D20240215T150055,D20240215T150055_IFCB010,41.325,-70.5667,D20240215T150055
3,D20240215T150055_IFCB010_00004,detritus,1.0000,2024-02-15,D20240215T150055,D20240215T150055_IFCB010,41.325,-70.5667,D20240215T150055
4,D20240215T150055_IFCB010_00005,detritus,0.9620,2024-02-15,D20240215T150055,D20240215T150055_IFCB010,41.325,-70.5667,D20240215T150055


In [15]:
import hypercoast

# Step 1: Login to NASA Earthdata
hypercoast.nasa_earth_login()# this will be your username and password for nasa_earth. you can get it easily


In [17]:
#Search for Chl-a data within a specific temporal range
temporal = ("2024-05-01", "2024-06-21")  # Adjust date range as needed
results = hypercoast.search_pace_chla(temporal=temporal)

# Step 3: Download the Chl-a data thi is the type three processed chl-a data
hypercoast.download_nasa_data(results, "chla")

# Print the search results to see what you found
print(results)


QUEUEING TASKS | :   0%|          | 0/47 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/47 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/47 [00:00<?, ?it/s]

[Collection: {'ShortName': 'PACE_OCI_L3M_CHL_NRT', 'Version': '2.0'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'SouthBoundingCoordinate': -90, 'WestBoundingCoordinate': -180, 'NorthBoundingCoordinate': 90, 'EastBoundingCoordinate': 180}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2024-05-01T00:00:00Z', 'EndingDateTime': '2024-05-01T23:59:59Z'}}
Size(MB): 2.894341468811035
Data: ['https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/PACE_OCI.20240501.L3m.DAY.CHL.V2_0.chlor_a.0p1deg.NRT.nc'], Collection: {'Version': '2.0', 'ShortName': 'PACE_OCI_L3M_CHL_NRT'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'NorthBoundingCoordinate': 90, 'WestBoundingCoordinate': -180, 'SouthBoundingCoordinate': -90, 'EastBoundingCoordinate': 180}]}}}
Temporal coverage: {'RangeDateTime': {'EndingDateTime': '2024-05-02T23:59:59Z', 'BeginningDateTime': '2024-05-02T00:00:00Z'}}
Size(MB): 2.658063888

In [18]:

from hypercoast import read_pace_chla, pace_chla_to_image

# Step 4: Read the downloaded data files into an array
files = "chla/*.nc"  # Adjust file path to where your data is saved
array = read_pace_chla(files)

# Step 5: Select data for a specific date range and calculate the average
flow_date = "2024-05-26"#these should meet with the IFCB dashbpard for one day, you can easily select more days to get more coverage 
selected_array = array.sel(date=slice(flow_date))


# Step 6: Convert the averaged array to an image format
single_image = pace_chla_to_image(selected_array)


In [23]:
#PLOT GROUP DISTRIBUTION WITH HYPERCOAST
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import ipywidgets as widgets
from hypercoast import Map

# Function to create genus plot for a given bin_date and location (with log-transformed Y-axis)
def create_genus_plot(species_data):
    fig, ax = plt.subplots(figsize=(15, 10))
    ax.bar(species_data['genus'], species_data['max_value'])
    ax.set_yscale('log')
    ax.set_title(f"Genus Distribution (Log Scale)", fontsize=20)
    ax.set_xlabel('Genus', fontsize=18)
    ax.set_ylabel('Max Value (log scale)', fontsize=18)
    ax.tick_params(axis='x', labelrotation=90, labelsize=12)

    # Convert the plot to a base64-encoded image
    buf = BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    plt.close(fig)
    buf.seek(0)
    
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    html_img = f'<img src="data:image/png;base64,{img_base64}" style="width:100%;">'
    return html_img

# Group data by latitude, longitude, and bin_date
locations = difcb.groupby(['latitude', 'longitude', 'bin_date'])

# Create the map centered on your stations
m = Map(center=[41.16341, -70.957489], zoom=8)
m.add_basemap("Hybrid")

# Add the averaged Chl-a raster layer to the map
m.add_raster(
    single_image,
    cmap="jet",
    vmin=-1,
    vmax=2,
    layer_name="Chlorophyll a",
    zoom_to_layer=False,
)

# Add genus distribution markers to the map for each unique bin_date and location
for (lat, lon, bin_date), location_data in locations:
    
    # Group data by genus for the specific bin_date and location
    genus_data = location_data.groupby('genus', as_index=False)['max_value'].sum()
    
    # Create genus distribution plot for the bin_date and location
    genus_plot_html = create_genus_plot(genus_data)
    
    # Construct the HTML for the popup
    popup_html = f"<strong>Station Info</strong><br>Location: {lat:.6f}, {lon:.6f}<br>Bin Date: {bin_date}<br>"
    popup_html += "<div>Genus Distribution:</div>"
    
    # Add the genus distribution plot directly
    popup_html += f'<div id="species_img">{genus_plot_html}</div>'
    
    # Create an HTML widget from the string and set the size of the popup
    popup_widget = widgets.HTML(value=popup_html)
    popup_container = widgets.Box([popup_widget], layout=widgets.Layout(width='600px', height='500px'))
    
    # Add the marker to the map
    location = (lat, lon)
    m.add_marker(location=location, popup=popup_container)

# Display the map with Chl-a raster and species markers
m


Map(center=[41.16341, -70.957489], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…